This notebook provides visaulization of public transport, groceris and hospitals.

In [ ]:
import folium

# Sample data for 5 properties
properties = [
    {
        "address": "(Leased) 3 Yarra Street, South Yarra VIC 3141",
        "geometry": "POINT (144.992828 -37.838173)",
        "closest_hospital_name": "Victorian Institute of Forensic Mental Health",
        "closest_hospital_coords": [-37.813134, 144.991197],
        "closest_train_station_name": "South Yarra Railway Station",
        "closest_train_station_coords": [-37.839376, 144.993003],
        "closest_tram_station_name": "127-South Yarra Railway Station/Toorak Rd",
        "closest_tram_station_coords": [-37.838353, 144.994321],
    },
    {
        "address": "004B/12 Albert Street, Hawthorn East VIC 3123",
        "geometry": "POINT (145.048408 -37.823472)",
        "closest_hospital_name": "Caritas Christi Hospice",
        "closest_hospital_coords": [-37.825043, 145.022962],
        "closest_train_station_name": "Auburn Railway Station",
        "closest_train_station_coords": [-37.828298, 145.024973],
        "closest_tram_station_name": "37-Tooronga Rd/Riversdale Rd",
        "closest_tram_station_coords": [-37.826370, 145.036127],
    },
    {
        "address": "04/949 Dandenong Road, Malvern East VIC 3145",
        "geometry": "POINT (145.048262 -37.878631)",
        "closest_hospital_name": "Caulfield Hospital",
        "closest_hospital_coords": [-37.887234, 145.041699],
        "closest_train_station_name": "Caulfield Railway Station",
        "closest_train_station_coords": [-37.885693, 145.040218],
        "closest_tram_station_name": "60-Burke Rd/Waverley Rd",
        "closest_tram_station_coords": [-37.878962, 145.043474],
    },
    {
        "address": "0711/333 Exhibition St, Melbourne VIC 3000",
        "geometry": "POINT (144.96834 -37.807831)",
        "closest_hospital_name": "St Vincent's Hospital",
        "closest_hospital_coords": [-37.804896, 144.975080],
        "closest_train_station_name": "Parliament Railway Station",
        "closest_train_station_coords": [-37.810392, 144.976847],
        "closest_tram_station_name": "8-Exhibition St/La Trobe St",
        "closest_tram_station_coords": [-37.808845, 144.973931],
    },
    {
        "address": "1 & 2/23 Koonawarra Street, Clayton VIC 3168",
        "geometry": "POINT (145.124589 -37.90863)",
        "closest_hospital_name": "Monash Medical Centre",
        "closest_hospital_coords": [-37.910538, 145.124970],
        "closest_train_station_name": "Clayton Railway Station",
        "closest_train_station_coords": [-37.911818, 145.134024],
        "closest_tram_station_name": "68-Carnegie/Truganini Rd",
        "closest_tram_station_coords": [-37.900752, 145.041162],
    },
]

map_center = [-37.850, 144.963] 
m = folium.Map(location=map_center, zoom_start=12)

train_icon_svg = '../data/raw/train.svg'  
tram_icon_svg = '../data/raw/tram-car.svg' 

for property in properties:
    lon, lat = map(float, property["geometry"].replace("POINT (", "").replace(")", "").split())
    
    folium.Marker(
        location=[lat, lon],
        popup=f"<strong>Property:</strong> {property['address']}",
        icon=folium.Icon(color='blue', icon='home')
    ).add_to(m)

    folium.Circle(
        location=[lat, lon],
        radius=500,  
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
    ).add_to(m)

    folium.Marker(
        location=property["closest_hospital_coords"],
        popup=(
            f"<strong>Hospital:</strong> {property['closest_hospital_name']}"
        ),
        icon=folium.Icon(color='red', icon='plus')
    ).add_to(m)

    folium.Marker(
        location=property["closest_train_station_coords"],
        popup=(
            f"<strong>Train Station:</strong> {property['closest_train_station_name']}"
        ),
        icon=folium.DivIcon(
            html=f'<div><img src="../notebooks/train.jpg" style="width:35px; height:35px;"></div>' 
        )
    ).add_to(m)

    folium.Marker(
        location=property["closest_tram_station_coords"],
        popup=(
            f"<strong>Tram Station:</strong> {property['closest_tram_station_name']}"
        ),
        icon=folium.DivIcon(
            html=f'<div><img src="{tram_icon_svg}" style="width:35px; height:35px;"></div>'  
        )
    ).add_to(m)

m.save("property_map_with_custom_icons.html")


In [30]:
m

In [36]:
import folium
import pandas as pd
import re

# Data as a string
data = """name,shop,addr:street,geometry
Yanakie General Store,supermarket,Meeniyan - Promontory Road,POINT (146.2076606 -38.8125072)
Coles,supermarket,Franklin Street,POINT (146.5398932 -38.1939842)
Coles,supermarket,George Street,POINT (146.3983912 -38.2390054)
Traralgon Asian Groceries,supermarket,Seymour Street,POINT (146.5404186 -38.1962636)
Coles,supermarket,Albert Street,POINT (146.25881765550164 -38.173613148356914)
ALDI,supermarket,Saviges Road,POINT (146.2579834290304 -38.17526979339021)
ALDI,supermarket,Argyle Street,POINT (146.5554809 -38.1925375)
Coles,supermarket,Marshalls Road,POINT (146.56222161743113 -38.17909798421704)
IGA,supermarket,Roy Street East,POINT (146.3165864 -36.3583955)
Aldi,supermarket,Chisholm Street,POINT (146.327161 -36.3563765)
Coles,supermarket,Ryley Street,POINT (146.3172286 -36.3599515)
Foodworks,supermarket,Myrtle Street,POINT (146.7266738 -36.562886)
ALDI,supermarket,Chisholm Street,POINT (146.32712855000003 -36.356370850000005)
Coles,supermarket,Clyde Street,POINT (146.72637195426594 -36.56143886107541)
IGA,supermarket,Conness Street,POINT (146.60809984999997 -36.151489500000004)
Ciccone's IGA Plus Liquor,supermarket,Towong Street,POINT (147.1792663 -36.2162184)
ALDI,supermarket,Elgin Boulevard,POINT (146.8815944 -36.119091)
Coles,supermarket,South Street,POINT (146.8888404 -36.1202037)
Eskdale IGA X-press plus Liquor,supermarket,Omeo Highway,POINT (147.2489774 -36.4645659)
Woolworths,supermarket,Gavan Street,POINT (146.9589197950966 -36.72727716024941)
Coles,supermarket,Cunninghame Street,POINT (147.0644793 -38.1069769)
IGA,supermarket,Tyers Street,POINT (147.0796469 -37.9663604)
ALDI,supermarket,Raymond Street,POINT (147.0660645 -38.1040078)
Foodworks,supermarket,Powerscourt Street,POINT (146.9858285920161 -37.95996968332763)
ALDI,supermarket,Nicholson Street,POINT (147.6279603030387 -37.82457966948586)
Coles,supermarket,Main Street,POINT (147.632703986698 -37.82514093268894)
ALDI,supermarket,Marine Parade,POINT (147.97418408041207 -37.88209617062416)
Foodworks,supermarket,Nicholson Street,POINT (148.4549913 -37.7079778)
Orbost IGA X-press plus Liquor,supermarket,Nicholson Street,POINT (148.4556723 -37.7050097)
Mirboo North IGA,supermarket,Ridgway,POINT (146.1585741 -38.4014473)
ALDI,supermarket,Bruce Street,POINT (145.9449685 -38.4786703)
Foodworks,supermarket,Reilly Street,POINT (145.7286408006974 -38.632874644614965)
Coles,supermarket,Palmerston Street,POINT (145.9324454 -38.1620606)
Poowong IGA X-Press,supermarket,Ranceby Road,POINT (145.7663866 -38.3426959)
IGA Neerim South Plus Liquor,supermarket,"Main Neerim Road, Neerim South",POINT (145.9541018 -38.0162591)
IGA Trafalgar,supermarket,McCrorey Street,POINT (146.153376 -38.2084944)
Coles,supermarket,Hope Street,POINT (145.8558181051272 -38.134356769988806)
IGA,supermarket,High Street,POINT (146.0901564 -37.0520648)
Foodworks,supermarket,High Street,POINT (146.0866578 -37.0515675)
Woolworths,supermarket,Smythe Street,POINT (145.9889959 -36.5495184)
ALDI,supermarket,Bridge Street East,POINT (145.9901922 -36.549889)
Coles,supermarket,Carrier Street,POINT (145.9858669829451 -36.549763168606354)
Woolworths,supermarket,Thompson Avenue,POINT (145.2397764 -38.4531708)
Coles,supermarket,Biggs Drive,POINT (145.5864483 -38.6037001)
ALDI,supermarket,Thompson Avenue,POINT (145.2385072835121 -38.450695440358515)
Woolworths,supermarket,Murray Street,POINT (145.58959411580378 -38.60456893554078)
Cowes IGA,supermarket,Thompson Avenue,POINT (145.239638362738 -38.456426103550434)
Coles,supermarket,Church Street,POINT (145.23843635392413 -38.45269202392654)
Coles,supermarket,Stud Road,POINT (145.2375525 -37.9208945)"""

lines = data.splitlines()
data_tuples = [tuple(line.split(',', 3)) for line in lines[1:]] 
df = pd.DataFrame(data_tuples, columns=lines[0].split(','))

df['longitude'] = df['geometry'].apply(lambda x: float(re.findall(r'[-+]?\d*\.\d+|\d+', x)[0]))
df['latitude'] = df['geometry'].apply(lambda x: float(re.findall(r'[-+]?\d*\.\d+|\d+', x)[1]))

map_center = [df['latitude'].mean(), df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=8)

svg_file_path = '../data/raw/cart.svg'

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']}<br>{row['addr:street']}",
        icon=folium.DivIcon(
            html=f'<img src="{svg_file_path}" style="width:30px; height:30px;">'  # Adjust size as needed
        )
    ).add_to(m)

m.save("supermarkets_map_with_img_icon.html")

m
